# Portfolio Tracker

In [1]:
from plutus.portfolio.trackers import BinanceTracker
from plutus.trade.exchanges import Binance
from plutus.configuration import config
from plutus import database as db
from plutus.models.enums import *
from plutus.models import *
from sqlmodel import Session, select, text, desc, update, func
from datetime import datetime, timedelta, timezone
import ccxt.async_support as ccxt
import sqlalchemy as sa
import pandas as pd
import numpy as np
import asyncio
import time
import json
import pytz

ModuleNotFoundError: No module named 'inflect'

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
with open('secrets/ftx.json') as fopen:
    ftx_api_key = json.loads(fopen.read())
    
with open('secrets/binance.json') as fopen:
    binance_api_key = json.loads(fopen.read())

## Binance

In [ ]:
binance = Binance(binance_api_key)

In [ ]:
binance_futures = BinanceFuturesBase('binancecoinm', binance_api_key)

In [ ]:
incomes = await binance_futures._fetch_incomes(
    'dapiPrivate_get_income',
    since=datetime(2021,10,1, tzinfo=timezone.utc),
    max_interval = timedelta(days=200)
)

In [ ]:
incomes1 = await binance.exchanges['usdm'].fetch_incomes(
    since=datetime(2021,10,1, tzinfo=timezone.utc)
)

In [ ]:
len(incomes)

In [ ]:
incomes1

In [ ]:
await binance.exchanges['usdm'].fetch_positions()

In [ ]:
await binance.exchanges['usdm'].api.fetch_positions()

In [ ]:
market = await binance.load_markets()

In [ ]:
my_spot_trades = await binance.fetch_my_trades(
    'BTCUSDT', exchange='spot', 
    since=datetime(2021,10,1, tzinfo=timezone.utc),
    # from_id=1,
)

In [ ]:
my_spot_trades

In [ ]:
await binance.fetch_asset_balance('usdm')

In [ ]:
fh = await binance.fetch_funding_history(since=datetime(2022,4,1))

In [ ]:
fh = pd.DataFrame(fh)

In [ ]:
fh['datetime'] = pd.to_datetime(fh['datetime']).dt.round('1h')

In [ ]:
since_dict = fh.groupby(['symbol'])['datetime'].min().to_dict()

In [ ]:
since_dict

In [ ]:
frh = await binance.fetch_funding_rate_history(
    symbol='AVAX/USDT', exchange='usdm',since=since
)

In [ ]:
frh = pd.DataFrame(frh)

In [ ]:
fh[fh['symbol'] == 'AVAX/USDT']

In [ ]:
frh['datetime'] = pd.to_datetime(frh['datetime']).dt.round('1h')

In [ ]:
frh = frh[['symbol', 'datetime', 'fundingRate']]

In [ ]:
fh.merge(frh, how='inner',on=['symbol', 'datetime'])

In [ ]:
binance.exchanges['coinm'].api.parse_incomes(funding_history)

In [ ]:
list(results)

In [ ]:
await binance.fetch_asset_balance()

In [ ]:
await binance.fetch_deposits(since=datetime(2021,10,1))

In [ ]:
await binance.fetch_withdrawals()

In [ ]:
await binance.fetch_my_trades(
    'ETH/USD', exchange='coinm', 
    since=datetime(2021,10,1, tzinfo=timezone.utc),
)

In [ ]:
await binance.fetch_convert_history(since=datetime(2021,10,1))

In [ ]:
await binance.fetch_ticker('BTCUSDT')

In [ ]:
await binance.fetch_funding_history(since=datetime(2021,10,1))

In [ ]:
await binance.fetch_c2c_trades(since=datetime(2021,10,1))

In [ ]:
await binance.exchanges['usdm'].api.fetch_positions()

In [ ]:
await binance.fetch_ticker('BTCUSDT')

In [ ]:
usdm_incomes = await binance.fetch_futures_income(exchange='usdm',since=datetime(2021,10,1))

In [ ]:
usdm_commissions = await binance.fetch_futures_commissions('usdm',since=datetime(2021,10,1))
coinm_commissions = await binance.fetch_futures_commissions('coinm',since=datetime(2021,10,1))

In [ ]:
usdm_commissions = pd.DataFrame(usdm_commissions)
coinm_commissions = pd.DataFrame(coinm_commissions)

In [ ]:
usdm_ids = usdm_commissions.groupby('symbol')['tradeId'].min().to_dict()
coinm_ids = coinm_commissions.groupby('symbol')['tradeId'].min().to_dict()

In [ ]:
usdm_ids

In [ ]:
coinm_ids

In [ ]:
my_usdm_trades = await binance.fetch_my_trades(
    'BTCUSDT', exchange='usdm', 
    from_id=1686975525
)

In [ ]:
my_coinm_trades = await binance.fetch_my_trades(
    'BTCUSD_PERP', exchange='coinm', 
    from_id=260029163
)

In [ ]:
my_spot_trades[0]

In [ ]:
my_usdm_trades[0]

In [ ]:
my_coinm_trades[0]

In [ ]:
tracker.process_my_trades('coinm', my_coinm_trades)

In [ ]:
help(tracker.process_my_trades)

In [ ]:
df = pd.DataFrame(my_usdm_trades)

In [ ]:
df['info'].apply(pd.Series)['side'].str.lower()

In [ ]:
tracker.process_spot_trades(my_trades)

In [ ]:
my_trades

In [ ]:
incomes = await binance.fetch_futures_income(since=datetime(2021,10,1))

In [ ]:
incomes

In [ ]:
await binance.close()

In [ ]:
with Session(db.engine) as session:
    debit_account = TAccount(id=2).get(session)

## Binance Tracker

In [ ]:
BASE_CURRENCY = 'USDT'
CASH_EQUIVALENTS = ['USDT', 'BUSD', 'USDC'] 

In [2]:
with Session(db.engine) as session:
    statement = (
        select(Account)
        .join(User)
        .join(Platform)
        .where(
            (User.name == 'Cheun Hong') &
            (Platform.name == 'binance')
        )
    )
    print(statement)
    results = session.exec(statement)
    account = results.first()

NameError: name 'Session' is not defined

In [3]:
tracker = BinanceTracker(binance_api_key, account_id = account.id)

NameError: name 'BinanceTracker' is not defined

In [ ]:
tracker.get_positions_df(AssetType.crypto_perp)

In [ ]:
# trades = await tracker.init_spot_balance()

In [ ]:
new_trades = await tracker.fetch_new_spot_trades()

In [ ]:
new_trades = await tracker.fetch_new_futures_trades('usdm')

In [ ]:
new_trades

In [ ]:
last_transacted = tracker.get_last_transacted_at(AssetType.crypto_perp)


In [ ]:
last_transacted

In [ ]:
comms = await tracker.binance.fetch_commissions(
    exchange='usdm'
)

In [ ]:
comms

In [ ]:
new_trades = await tracker.binance.fetch_my_trades(
    'BTCUSDT', 'usdm', from_id=2154011959
)

In [ ]:
new_trades

In [ ]:
new_trades

In [ ]:
new_funding_fees = await tracker.fetch_new_funding_fees('usdm')

In [ ]:
new_funding_fees

In [ ]:
new_funding_fees['info'].apply(pd.Series)

In [ ]:
new_funding_fees['info_y'].apply(pd.Series)

In [ ]:
new_trades

In [ ]:
processed_my_trades = tracker.process_my_trades('spot', new_trades)

In [ ]:
processed_my_trades = tracker.process_my_trades('usdm', new_trades)

In [ ]:
processed_my_trades

In [ ]:
processed_my_trades

In [ ]:
await binance.fetch_futures_commissions(since=last_transacted)

In [ ]:
start = time.time()
await tracker.record_spot_trades()
print('Used', time.time() - start, 'seconds')

In [ ]:
start = time.time()
await tracker.record_futures_trades()
print('Used', time.time() - start, 'seconds')

In [ ]:
start = time.time()
await tracker.record_funding_history()
print('Used', time.time() - start, 'seconds')

In [ ]:
await tracker.get_current_spot_balance()

In [ ]:
tracker.get_spot_balance()

In [ ]:
await tracker.get_spot_balance_discrepancy()

In [ ]:
await tracker.get_spot_balance_discrepancy()

In [ ]:
await tracker.update_spot_positions()

In [ ]:
await tracker.close()

### DB

In [ ]:
db.initdb()

## Web 3

In [ ]:
from web3 import Web3
import requests, time, json

In [ ]:
bsc = 'https://bsc-dataseed.binance.org/'
web3 = Web3(Web3.HTTPProvider(bsc))
print(web3.isConnected())

In [ ]:
address = '0xD40046b6351D6C9d193a337d0A9E645fA7f22D2a'
balance = web3.eth.get_balance(address)
print(balance)

In [ ]:
result = web3.fromWei(balance,'ether')
print(result)

In [ ]:
MyAddress = '0xD40046b6351D6C9d193a337d0A9E645fA7f22D2a'
TokenAddress = '0xe9e7cea3dedca5984780bafc599bd69add087d56'

url_eth = 'https://api.bscscan.com/api'
contract_address = web3.toChecksumAddress(TokenAddress)
API_ENDPOINT = (
    url_eth
    + '?module=contract&action=getabi&address='
    + str(contract_address)
)
r = requests.get(url = API_ENDPOINT)
response = r.json()
abi = json.loads(response['result'])

In [ ]:
contract = web3.eth.contract(address=contract_address, abi=abi)
totalSupply = contract.functions.totalSupply().call()
print(totalSupply)
print(contract.functions.name().call())
print(contract.functions.symbol().call())
address = web3.toChecksumAddress(MyAddress)
balance=contract.functions.balanceOf(address).call()
print(web3.fromWei(balance, 'ether'))

## Testing SQLModel

In [ ]:
from portfi.models.enums import AssetType
from portfi.models.base import BaseModel
from portfi import database as db
from sqlmodel import (
    SQLModel, MetaData, Field, Column, ForeignKey, 
    Index, Session, String, Enum, text, select,
    create_engine, ForeignKey, Relationship
)
from sqlalchemy.dialects.mysql import INTEGER, TIMESTAMP
from sqlalchemy.ext.declarative import declared_attr
from google.cloud import secretmanager
from datetime import datetime
from typing import Optional, List
import sqlalchemy as sa
import pandas as pd
import json
import re

In [ ]:
class Test(BaseModel, table=True):
    __table_args__ = (
        Index('name_idx', 'name', unique=True),
        # {'extend_existing': True},
    )
    name: str = Field(sa_column=Column(String(10)))
    asset_type: Optional[AssetType] = Field(sa_column=Column(Enum(AssetType)))

In [ ]:
test = Test(name='b')

In [ ]:
dir(test)

In [ ]:
test.

In [ ]:
with Session(db.engine) as session:
    test = Test(name='b')
    test.add(session)
    session.commit()

In [ ]:
db.initdb()

## Coingecko API

In [ ]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [ ]:
cg.ping()

In [ ]:
cg.get_coins_categories_list()

In [ ]:
cg.get_coins_categories()


In [ ]:
cg.get_coins_list()


In [ ]:
len('currency_exchanges')

In [ ]:
type(datetime.now())

In [ ]:
from datetime import timedelta

In [ ]:
type(timedelta(minutes=7))

In [ ]:
from tradesk.crypto.exchanges import Binance

In [ ]:
help(Binance)

In [ ]:
from typing_extensions import Literal

a = Literal['a', 'b']

In [ ]:
a.__args__ = ('a', 'b', 'c')

In [ ]:
a

In [ ]:
from google.cloud import secretmanager
import json

In [ ]:
client = secretmanager.SecretManagerServiceClient()

In [ ]:
secret = 'projects/235345915907/secrets/db-cred/versions/1'
payload = json.loads(
    client.access_secret_version(name = secret).payload.data.decode('utf-8')
)

In [ ]:
payload

In [ ]:
from typing_extensions import Literal

In [ ]:
a = Literal['a', 'b']

In [ ]:
dir(a)

In [ ]:
'b' in a.__args__

In [ ]:
def infinite_sequence():
    num = 0
    while True:
        yield num
        print('plus 1')
        num += 1

In [ ]:
for i in infinite_sequence():
    print(i)
    if i > 10:
        break